# import libraries

In [1]:
import os
import copy
import random
import pyvista as pv
import trimesh as tm
import numpy as np
import topogenesis as tg

# import the environment

In [2]:
# load context file. Currently set as Rotterdam area from spatial computing
context_path = os.path.relpath('../data/immediate_context.obj')
context = tm.load(context_path)

# create macro voxels

In [3]:
# base of the 'rubik's cube'
unit = 45
division = 3
macrosize = int(unit/division)
base = tm.creation.box((unit,unit,unit))
m_vox_size = (macrosize,macrosize,macrosize)
base.apply_translation([60, -50, unit/2])

<trimesh.Trimesh(vertices.shape=(8, 3), faces.shape=(12, 3))>

In [4]:
# for loop to create initial macro voxels
vox_macro = []
for x in range(-1,2):
    xval = x*macrosize
    for y in range(-1,2):
        yval = y*macrosize
        for z in range(-1,2):
            zval = z*macrosize
            box = tm.creation.box(m_vox_size, transform=tm.transformations.translation_matrix((xval,yval,zval)))
            box.apply_translation([60, -50, unit/2]) # move objects to acceptable starting position
            vox_macro.append(box)

In [5]:
# testing configuration:

# create random configuration
config_rand = [random.randint(0,1) for i in range(26)]

# voxels of the configuration
my_config = [vox_macro[i] for i in range(26) if config_rand[i] == 1]
config_index = [i for i in range(26) if config_rand[i] == 1]

In [6]:
# creating union is major time cost
configuration = tm.boolean.union(my_config)

In [7]:
# prepping ladybug
import ladybug
from ladybug.epw import EPW
from ladybug.sunpath import Sunpath
epw_data = EPW('../data/NLD_Amsterdam.062400_IWEC.epw')
write_path = os.path.relpath('../output')
RottAmsterdam = epw_data.location # context is Rotterdam, information used is Amsterdam TODO:unify
# sp = Sunpath.from_location(RottAmsterdam)
# sun = sp.calculate_sun(month=11, day=15, hour=11.0)
# ladybug.skymodel, honeybee.radiance --> how to incorporate (surrounding) geometry

# visualizing


In [8]:
# convert to trimesh definition, taken from spatial computing workshop
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter

# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the base mesh: light blue
p.add_mesh(tri_to_pv(context), color='#abd8ff')
p.add_mesh(tri_to_pv(base), color='#804080', opacity=0.2)
p.add_mesh(tri_to_pv(configuration), color='#104080')


# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(785.6075686833789, 708.1911636833788, 743.2184808333789),
 (65.08283250000001, -12.333572500000002, 22.69374465),
 (0.0, 0.0, 1.0)]